In [1]:
# import tensorflow as tsf
# ! pip install -qU transformers accelerate langchain==0.0.174 xformers sentencepiece
# ! pip install tensorflow
import numpy
import accelerate
import json
import transformers
from torch import cuda, bfloat16
import torch
from llama_cpp import Llama

d:\Chrome Download\Anaconda\ana\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
binary_path: d:\Chrome Download\Anaconda\ana\lib\site-packages\bitsandbytes\cuda_setup\libbitsandbytes_cuda116.dll
CUDA SETUP: Loading binary d:\Chrome Download\Anaconda\ana\lib\site-packages\bitsandbytes\cuda_setup\libbitsandbytes_cuda116.dll...


In [2]:
print("torch, cuda is avaliable:", torch.cuda.is_available())
print("be remined dont download CPU version: (xxx+cpu)", torch.__version__)
print(f"your torch version: {torch.version.cuda}")
device=f"{cuda.current_device()}" if cuda.is_available() else "cpu"

print(f"how many GPU you'd have: {cuda.device_count()}\n", "Now show the device number/first device name: ", device, cuda.get_device_name(0))

torch, cuda is avaliable: True
be remined dont download CPU version: (xxx+cpu) 2.0.1
your torch version: 11.7
how many GPU you'd have: 1
 Now show the device number/first device name:  0 NVIDIA GeForce RTX 2060


In [3]:
# from huggingfacehub import login
straight_path=r"D:\LLama2\llama_hugging\pytorch_model.bin"
llm=Llama(straight_path)

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | VSX = 0 | 


In [10]:
output = llm("Q: Can you be my wife. A: ", max_tokens=512, stop=["Q:", "\n"], echo=True)

Llama.generate: prefix-match hit


In [11]:
print(json.dumps(output, indent=4))

{
    "id": "cmpl-b3677b3c-82fa-41bb-a5f9-f0f977176361",
    "object": "text_completion",
    "created": 1693063785,
    "model": "D:\\LLama2\\llama_hugging\\pytorch_model.bin",
    "choices": [
        {
            "text": "Q: Can you be my wife. A: \ud83d\ude36 Sorry, I cannot fulfill that request as I am just a computer program and do not have the ability to form romantic relationships or engage in marriage. My purpose is to assist with tasks and answer questions to the best of my ability based on my training and knowledge. Is there anything else I can help you with?",
            "index": 0,
            "logprobs": null,
            "finish_reason": "stop"
        }
    ],
    "usage": {
        "prompt_tokens": 12,
        "completion_tokens": 69,
        "total_tokens": 81
    }
}


In [6]:
# start to use tensorflow frame load model -- this stage may involved to download new model
my_store_path=r"D:\LLama2\llama_hugging"
llama_repo="TheBloke/Llama-2-13B-chat-GGML"
config = transformers.AutoConfig.from_pretrained(my_store_path)

In [7]:
# ! conda list torch
# print(config)
with accelerate.init_empty_weights():
    empty_model=transformers.AutoModelForCausalLM.from_config(config)

device_map=accelerate.infer_auto_device_map(empty_model, max_memory={0: "4GiB", "cpu": "6GiB"}) # , 
allocation=json.dumps(device_map, indent=4)
print(allocation)

{
    "model.embed_tokens": 0,
    "model.layers.0": 0,
    "model.layers.1": 0,
    "model.layers.2": 0,
    "model.layers.3.self_attn": 0,
    "model.layers.3.mlp.gate_proj": 0,
    "model.layers.3.mlp.up_proj": 0,
    "model.layers.3.mlp.down_proj": "cpu",
    "model.layers.3.mlp.act_fn": "cpu",
    "model.layers.3.input_layernorm": "cpu",
    "model.layers.3.post_attention_layernorm": "cpu",
    "model.layers.4": "cpu",
    "model.layers.5": "cpu",
    "model.layers.6": "cpu",
    "model.layers.7": "cpu",
    "model.layers.8": "cpu",
    "model.layers.9": "cpu",
    "model.layers.10": "cpu",
    "model.layers.12": "disk",
    "model.layers.13": "disk",
    "model.layers.14": "disk",
    "model.layers.15": "disk",
    "model.layers.16": "disk",
    "model.layers.17": "disk",
    "model.layers.18": "disk",
    "model.layers.19": "disk",
    "model.layers.20": "disk",
    "model.layers.21": "disk",
    "model.layers.22": "disk",
    "model.layers.23": "disk",
    "model.layers.24": "d

In [23]:
model=transformers.LlamaForCausalLM.from_pretrained(my_store_path, device_map=device_map, offload_folder="/tmp/.offload",
        load_in_8bit=True, llm_int8_enable_fp32_cpu_offload=True) 

OSError: Unable to load weights from pytorch checkpoint file for 'D:\LLama2\llama_hugging\pytorch_model.bin' at 'D:\LLama2\llama_hugging\pytorch_model.bin'. If you tried to load a PyTorch model from a TF 2.0 checkpoint, please set from_tf=True.

In [ ]:
import sys
sys.exit()